# Imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [51]:
import datetime
import pandas as pd
import seaborn as sns
import numpy as np
import datetime as dt

from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

import ngboost as ngb
import xgboost as xgb
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin

# Data Load

In [4]:
df = pd.read_pickle('/Users/lj/code/5pacepenguin/nba_betting_analysis/nba_betting_analysis/backend/data/pkl/scaled_boxscore_advanced_rolling_players.pkl')
df

,PLAYER_ID,TEAM_ID,TEAM_ABBREVIATION,GAME_DATE,PLAYER_NAME,PLUS_MINUS,AST_PCT,AST_RATIO,AST_TOV,DEF_RATING,...,OFF_RATING,OREB_PCT,PACE,PIE,POSS,PTS,REB_PCT,TM_TOV_PCT,TS_PCT,USG_PCT
0,1713,1610612737,ATL,2018-12-03,Vince Carter,-14.9,0.123266,0.169653,0.042814,0.578202,...,0.587002,0.050330,0.014105,0.542849,0.347285,0.421875,0.228959,0.072430,0.431366,0.376921
1,1713,1610612737,ATL,2018-12-05,Vince Carter,-14.8,0.132462,0.147789,0.042814,0.546110,...,0.573205,0.064214,0.013059,0.548627,0.383484,0.437500,0.231373,0.072430,0.465736,0.384718
2,1713,1610612737,ATL,2018-12-08,Vince Carter,-13.9,0.147144,0.161434,0.057086,0.548544,...,0.577199,0.083652,0.013363,0.554863,0.373303,0.437500,0.249472,0.090059,0.499260,0.418802
3,1713,1610612737,ATL,2018-12-12,Vince Carter,-13.9,0.125686,0.163571,0.014271,0.554043,...,0.561224,0.065255,0.014036,0.548169,0.368778,0.445312,0.231071,0.070731,0.447864,0.404767
4,1713,1610612737,ATL,2018-12-14,Vince Carter,-11.5,0.125686,0.163571,0.014271,0.595691,...,0.566125,0.047900,0.013775,0.547654,0.367647,0.474609,0.221418,0.070731,0.422377,0.418133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100079,1631323,1610612762,UTA,2023-02-23,Simone Fontecchio,0.6,0.092611,0.167352,0.071357,0.566844,...,0.706635,0.133287,0.012827,0.522883,0.263575,0.759766,0.171342,0.267842,0.421003,0.387391
100080,1631323,1610612762,UTA,2023-02-25,Simone Fontecchio,2.3,0.107293,0.190860,0.071357,0.579735,...,0.770627,0.152725,0.012945,0.538387,0.277149,0.761719,0.182202,0.214741,0.455690,0.367788
100081,1631323,1610612762,UTA,2023-02-28,Simone Fontecchio,-0.8,0.128751,0.218313,0.080919,0.630758,...,0.761732,0.152725,0.012803,0.522654,0.319005,0.699219,0.139065,0.267842,0.402813,0.317665
100082,1631323,1610612762,UTA,2023-03-03,Simone Fontecchio,-5.3,0.139561,0.228670,0.095191,0.612729,...,0.773260,0.096147,0.011583,0.523112,0.369910,0.666016,0.080543,0.139550,0.305626,0.312542


In [5]:
df.dtypes

PLAYER_ID                     int64
TEAM_ID                       int64
TEAM_ABBREVIATION            object
GAME_DATE            datetime64[ns]
PLAYER_NAME                category
PLUS_MINUS                  float64
AST_PCT                     float64
AST_RATIO                   float64
AST_TOV                     float64
DEF_RATING                  float64
DREB_PCT                    float64
EFG_PCT                     float64
MIN                         float64
NET_RATING                  float64
OFF_RATING                  float64
OREB_PCT                    float64
PACE                        float64
PIE                         float64
POSS                        float64
PTS                         float64
REB_PCT                     float64
TM_TOV_PCT                  float64
TS_PCT                      float64
USG_PCT                     float64
dtype: object

# Model

## Train-test split

In [6]:
# taken from neural_net.ipynb
train_test_split_date = '2022-04-27 00:00:00'
X_train = df.loc[df.GAME_DATE < train_test_split_date, ~df.columns.isin(['PLUS_MINUS'])]
X_test = df.loc[df.GAME_DATE > train_test_split_date, ~df.columns.isin(['PLUS_MINUS'])]
y_train = df.loc[df.GAME_DATE < train_test_split_date, ['PLUS_MINUS']]
y_test = df.loc[df.GAME_DATE > train_test_split_date, ['PLUS_MINUS']]

In [7]:
X_train, X_test, y_train, y_test

(       PLAYER_ID     TEAM_ID TEAM_ABBREVIATION  GAME_DATE   PLAYER_NAME  \
 0           1713  1610612737               ATL 2018-12-03  Vince Carter   
 1           1713  1610612737               ATL 2018-12-05  Vince Carter   
 2           1713  1610612737               ATL 2018-12-08  Vince Carter   
 3           1713  1610612737               ATL 2018-12-12  Vince Carter   
 4           1713  1610612737               ATL 2018-12-14  Vince Carter   
 ...          ...         ...               ...        ...           ...   
 98794    1630846  1610612760               OKC 2022-03-28  Olivier Sarr   
 98795    1630846  1610612760               OKC 2022-03-30  Olivier Sarr   
 98796    1630846  1610612760               OKC 2022-04-01  Olivier Sarr   
 98797    1630846  1610612760               OKC 2022-04-03  Olivier Sarr   
 98798    1630846  1610612760               OKC 2022-04-05  Olivier Sarr   
 
         AST_PCT  AST_RATIO   AST_TOV  DEF_RATING  DREB_PCT  ...  OFF_RATING  \
 0    

In [16]:
X_train_catless = X_train.loc[:, ~X_train.columns.isin(['GAME_DATE', 'PLAYER_ID', 'PLAYER_NAME'])]
X_test_catless = X_test.loc[:, ~X_test.columns.isin(['GAME_DATE', 'PLAYER_ID', 'PLAYER_NAME'])]
X_test_catless

,TEAM_ID,TEAM_ABBREVIATION,AST_PCT,AST_RATIO,AST_TOV,DEF_RATING,DREB_PCT,EFG_PCT,MIN,NET_RATING,OFF_RATING,OREB_PCT,PACE,PIE,POSS,PTS,REB_PCT,TM_TOV_PCT,TS_PCT,USG_PCT
456,1610612747,LAL,0.432720,0.262206,0.259027,0.650410,0.322285,0.604447,0.841656,0.544617,0.724426,0.093023,0.008474,0.608410,0.822398,0.585938,0.308296,0.230459,0.639594,0.730675
457,1610612747,LAL,0.448854,0.273385,0.230484,0.665915,0.315730,0.607013,0.797485,0.533373,0.722974,0.093023,0.009722,0.603890,0.787330,0.572266,0.302262,0.212829,0.638959,0.710403
458,1610612747,LAL,0.429171,0.266974,0.223348,0.614712,0.331086,0.629250,0.784186,0.560646,0.712807,0.106907,0.010254,0.610526,0.776018,0.576172,0.323680,0.217502,0.658206,0.695032
459,1610612747,LAL,0.400774,0.250041,0.214785,0.620391,0.339888,0.589053,0.729658,0.548086,0.699555,0.115932,0.010094,0.604405,0.720588,0.542969,0.333032,0.207519,0.623308,0.714413
460,1610612747,LAL,0.440626,0.262371,0.226202,0.608852,0.370974,0.601240,0.725144,0.554306,0.697377,0.154460,0.010898,0.613844,0.726244,0.552734,0.371342,0.200935,0.639065,0.719537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100079,1610612762,UTA,0.092611,0.167352,0.071357,0.566844,0.141199,0.415758,0.251965,0.588337,0.706635,0.133287,0.012827,0.522883,0.263575,0.759766,0.171342,0.267842,0.421003,0.387391
100080,1610612762,UTA,0.107293,0.190860,0.071357,0.579735,0.141199,0.429121,0.264539,0.622010,0.770627,0.152725,0.012945,0.538387,0.277149,0.761719,0.182202,0.214741,0.455690,0.367788
100081,1610612762,UTA,0.128751,0.218313,0.080919,0.630758,0.094382,0.375668,0.305364,0.582356,0.761732,0.152725,0.012803,0.522654,0.319005,0.699219,0.139065,0.267842,0.402813,0.317665
100082,1610612762,UTA,0.139561,0.228670,0.095191,0.612729,0.045880,0.277635,0.358884,0.601974,0.773260,0.096147,0.011583,0.523112,0.369910,0.666016,0.080543,0.139550,0.305626,0.312542


In [166]:
dtrain_reg = DMatrix(X_train_dateless, y_train, enable_categorical=True)
dtest_reg = DMatrix(X_test_dateless, y_test, enable_categorical=True)

## 3.2 NGBoost with Hyperopt

In [45]:
b1 = DecisionTreeRegressor(criterion='friedman_mse', max_depth=20)
b2 = DecisionTreeRegressor(criterion='friedman_mse', max_depth=35)
b3 = DecisionTreeRegressor(criterion='friedman_mse', max_depth=50)

In [48]:
ngb.scores.LogScore

ngboost.scores.LogScore

In [65]:
space = {
    'Dist' : hp.choice('Dist', [ngb.distns.Normal]),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.01),
    'n_estimators' : hp.choice('n_estimators', range(20, 200, 5)),
    #'gamma' : hp.quniform('gamma', 0, 0.50, 0.01),
    #'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
    'minibatch_frac' : hp.quniform('minibatch_frac', 0.1, 1, 0.01),
    'Score' : hp.choice('Score', [ngb.scores.LogScore]),
    'col_sample' : hp.quniform('col_sample', 0.1, 1.0, 0.01),
    'Base': hp.choice('Base', [b1, b2, b3])
}

In [13]:
base = xgb.XGBRegressor()
base.load_model('/Users/lj/code/5pacepenguin/nba_betting_analysis/nba_betting_analysis/backend/models/hyperopted_gbdt_players_33estimators')
base

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.53, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.35000000000000003, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.06, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=50,
             max_leaves=None, min_child_weight=6.0, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=None, ...)

In [64]:
X_train.describe()

,PLAYER_ID,TEAM_ID,AST_PCT,AST_RATIO,AST_TOV,DEF_RATING,DREB_PCT,EFG_PCT,MIN,NET_RATING,OFF_RATING,OREB_PCT,PACE,PIE,POSS,PTS,REB_PCT,TM_TOV_PCT,TS_PCT,USG_PCT
count,7.970800e+04,7.970800e+04,79708.000000,79708.000000,79708.000000,79708.000000,79708.000000,79708.000000,79708.000000,79708.000000,79708.000000,79708.000000,79708.000000,79708.000000,79708.000000,79708.000000,79708.000000,79708.000000,79708.000000,79708.000000
mean,1.019341e+06,1.610613e+09,0.227147,0.270407,0.151329,0.587824,0.256084,0.530130,0.522678,0.558884,0.683243,0.140036,0.010450,0.572093,0.516615,0.580998,0.267454,0.213144,0.558146,0.390102
std,7.107855e+05,8.500417e+00,0.148640,0.135253,0.131200,0.076285,0.113695,0.120786,0.204998,0.070835,0.088492,0.121796,0.015361,0.033038,0.199153,0.115368,0.129764,0.096079,0.115660,0.127350
min,1.713000e+03,1.610613e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022787,0.000000,0.000000,0.000302,0.000000,0.000000,0.011719,0.000000,0.000000,0.000000,0.000000
25%,2.030980e+05,1.610613e+09,0.116328,0.171626,0.052376,0.546020,0.173408,0.465362,0.364406,0.519617,0.638831,0.053801,0.007786,0.556350,0.363122,0.503906,0.171342,0.150170,0.497039,0.298953
50%,1.627732e+06,1.610613e+09,0.186189,0.249877,0.114172,0.592085,0.234831,0.540411,0.536372,0.563397,0.692748,0.099965,0.009396,0.571739,0.530543,0.583984,0.236501,0.203696,0.570960,0.374471
75%,1.629006e+06,1.610613e+09,0.306551,0.350485,0.216498,0.635536,0.321348,0.606692,0.694152,0.604486,0.741150,0.190559,0.011195,0.588272,0.683258,0.660156,0.336953,0.263594,0.633249,0.466919
max,1.630846e+06,1.610613e+09,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.910407,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.933032,1.000000,1.000000,1.000000


In [66]:
def objective(space):

    #warnings.filterwarnings(action='ignore', category=DeprecationWarning)
    hyperopt_regressor = ngb.NGBRegressor(n_estimators = space['n_estimators'],
                            Dist = space['Dist'],
                            #max_depth = int(space['max_depth']),
                            learning_rate = space['learning_rate'],
                            #gamma = space['gamma'],
                            #min_child_weight = space['min_child_weight'],
                            minibatch_frac = space['minibatch_frac'],
                            col_sample = space['col_sample'],
                            Score = space['Score'],
                            Base = space['Base'],
                            early_stopping_rounds = 3
                            )
    
    hyperopt_regressor.fit(X_train.loc[:, ~X_train.columns.isin(['GAME_DATE', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION'])], 
                           np.ravel(y_train))

    # Applying k-Fold Cross Validation
    mses = cross_val_score(estimator = hyperopt_regressor, 
                           X = X_train.loc[:, ~X_train.columns.isin(['GAME_DATE', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION'])],
                           y = np.ravel(y_train), cv = 5, n_jobs=-1, scoring=make_scorer(mean_squared_error)
                          )
    
    CrossValMSE = mses.mean()

    print("CrossValMSE: ", CrossValMSE)

    return{'loss': CrossValMSE, 'status': STATUS_OK }

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)

print("Best: ", best)

[iter 0] loss=3.3075 val_loss=3.1890 scale=2.0000 norm=10.6727                  
== Early stopping achieved.                                                     
== Best iteration / VAL0 (val_loss=3.1890)                                      
CrossValMSE:                                                                    
53.66968558260976                                                               
[iter 0] loss=3.3029 val_loss=3.1887 scale=1.0000 norm=5.3234                   
== Early stopping achieved.                                                     
== Best iteration / VAL1 (val_loss=3.0908)                                      
CrossValMSE:                                                                    
37.843363441819896                                                              
[iter 0] loss=3.3084 val_loss=3.2001 scale=2.0000 norm=10.6653                  
== Early stopping achieved.                                                     
== Best iteration / VAL18 (v

/opt/homebrew/lib/python3.10/site-packages/ngboost/distns/normal.py:71: RuntimeWarning: overflow encountered in square
  self.var = self.scale**2


[iter 0] loss=3.3070 val_loss=3.3044 scale=2.0000 norm=10.6830                  
== Early stopping achieved.                                                     
== Best iteration / VAL0 (val_loss=3.3044)                                      
CrossValMSE:                                                                    
40.50987366039145                                                               
[iter 0] loss=3.3090 val_loss=3.2289 scale=2.0000 norm=10.6708                  
== Early stopping achieved.                                                     
== Best iteration / VAL25 (val_loss=2.7560)                                     
CrossValMSE:                                                                    
20.310413561124044                                                              
[iter 0] loss=3.3087 val_loss=3.1056 scale=2.0000 norm=10.7019                  
== Early stopping achieved.                                                     
== Best iteration / VAL3 (va

/opt/homebrew/lib/python3.10/site-packages/ngboost/distns/normal.py:71: RuntimeWarning: overflow encountered in square
  self.var = self.scale**2


[iter 0] loss=3.3081 val_loss=3.1146 scale=2.0000 norm=10.6853                  
== Early stopping achieved.                                                     
== Best iteration / VAL5 (val_loss=2.8722)                                      
CrossValMSE:                                                                    
23.264980837583284                                                              
[iter 0] loss=3.3065 val_loss=3.1527 scale=1.0000 norm=5.3395                   
== Early stopping achieved.                                                     
== Best iteration / VAL3 (val_loss=2.9591)                                      
CrossValMSE:                                                                    
22.626757774963753                                                              
[iter 0] loss=3.3073 val_loss=3.2870 scale=1.0000 norm=5.3385                   
== Early stopping achieved.                                                     
== Best iteration / VAL34 (v

...

CrossValMSE:                                                                    
83.2870126922503                                                                
[iter 0] loss=3.3073 val_loss=3.1983 scale=1.0000 norm=5.3391                   
== Early stopping achieved.                                                     
== Best iteration / VAL1 (val_loss=3.1304)                                      
CrossValMSE:                                                                    
26.127528817919245                                                              
[iter 0] loss=3.2988 val_loss=3.2737 scale=2.0000 norm=10.6122                  
== Early stopping achieved.                                                     
== Best iteration / VAL78 (val_loss=2.6259)                                     
CrossValMSE:                                                                    
17.361750726208005                                                              
[iter 0] loss=3.3194 val_los

[iter 0] loss=3.3136 val_loss=3.1314 scale=2.0000 norm=10.7450
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=3.1314)
[iter 0] loss=3.3132 val_loss=3.2253 scale=1.0000 norm=5.3780
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=3.2253)
[iter 0] loss=3.3118 val_loss=3.3003 scale=2.0000 norm=10.7128
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=3.3003)
[iter 0] loss=3.3109 val_loss=3.1892 scale=2.0000 norm=10.7125
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=3.1892)
[iter 0] loss=3.3192 val_loss=3.1774 scale=2.0000 norm=10.7938
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=3.1774)
[iter 0] loss=3.3148 val_loss=3.2505 scale=2.0000 norm=10.7503
== Early stopping achieved.
== Best iteration / VAL27 (val_loss=2.7322)
[iter 0] loss=3.3187 val_loss=3.1484 scale=2.0000 norm=10.7950
== Early stopping achieved.
== Best iteration / VAL5 (val_loss=2.9413)
[iter 0] loss=3.3240 val_loss=3.1789 scale=1.0000 norm=5.4181


[iter 0] loss=3.2985 val_loss=3.2064 scale=2.0000 norm=10.5999                  
== Early stopping achieved.                                                     
== Best iteration / VAL15 (val_loss=2.8479)                                     
CrossValMSE:                                                                    
21.92862309027281                                                               
[iter 0] loss=3.2978 val_loss=3.2216 scale=2.0000 norm=10.5690                  
== Early stopping achieved.                                                     
== Best iteration / VAL4 (val_loss=3.0293)                                      
CrossValMSE:                                                                    
25.812626203815192                                                              
[iter 0] loss=3.3068 val_loss=3.1161 scale=2.0000 norm=10.6689                  
== Early stopping achieved.                                                     
== Best iteration / VAL1 (va

CrossValMSE:                                                                    
52.09122801135529                                                               
[iter 0] loss=3.3043 val_loss=3.1139 scale=2.0000 norm=10.6390                  
== Early stopping achieved.                                                     
== Best iteration / VAL1 (val_loss=3.0662)                                      
CrossValMSE:                                                                    
37.42851879992906                                                               
[iter 0] loss=3.3012 val_loss=3.1371 scale=2.0000 norm=10.6230                  
== Early stopping achieved.                                                     
== Best iteration / VAL3 (val_loss=3.0512)                                      
CrossValMSE:                                                                    
25.50081726057204                                                               
[iter 0] loss=3.3078 val_los

[iter 0] loss=3.3080 val_loss=3.1350 scale=2.0000 norm=10.7092
== Early stopping achieved.
== Best iteration / VAL4 (val_loss=2.9610)
[iter 0] loss=3.3211 val_loss=3.1675 scale=2.0000 norm=10.7673
== Early stopping achieved.
== Best iteration / VAL5 (val_loss=2.9697)
[iter 0] loss=3.3043 val_loss=3.2554 scale=2.0000 norm=10.6662
== Early stopping achieved.
== Best iteration / VAL24 (val_loss=2.7894)
[iter 0] loss=3.3214 val_loss=3.2346 scale=2.0000 norm=10.8197
== Early stopping achieved.
== Best iteration / VAL10 (val_loss=2.9507)
[iter 0] loss=3.2679 val_loss=3.0889 scale=2.0000 norm=10.3135
== Early stopping achieved.
== Best iteration / VAL4 (val_loss=2.9196)
[iter 0] loss=3.3175 val_loss=3.2228 scale=2.0000 norm=10.7558
== Early stopping achieved.
== Best iteration / VAL19 (val_loss=2.8124)
[iter 0] loss=3.3044 val_loss=3.3454 scale=2.0000 norm=10.6360
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=3.3454)
[iter 0] loss=3.3174 val_loss=3.1861 scale=2.0000 norm=10.7

In [67]:
best

{'Base': 0,
 'Dist': 0,
 'Score': 0,
 'col_sample': 1.0,
 'learning_rate': 0.01,
 'minibatch_frac': 0.21,
 'n_estimators': 31}

[iter 0] loss=3.3203 val_loss=3.3463 scale=2.0000 norm=10.8171
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=3.3463)
[iter 0] loss=3.3017 val_loss=3.2555 scale=2.0000 norm=10.6058
== Early stopping achieved.
== Best iteration / VAL31 (val_loss=2.8408)
[iter 0] loss=3.2732 val_loss=3.1744 scale=2.0000 norm=10.3723
== Early stopping achieved.
== Best iteration / VAL7 (val_loss=2.7911)
[iter 0] loss=3.3082 val_loss=3.2331 scale=2.0000 norm=10.6809
== Early stopping achieved.
== Best iteration / VAL24 (val_loss=2.7350)
[iter 0] loss=3.3034 val_loss=3.1074 scale=2.0000 norm=10.6398
== Early stopping achieved.
== Best iteration / VAL4 (val_loss=2.8704)
[iter 0] loss=3.3234 val_loss=3.3070 scale=2.0000 norm=10.8451
[iter 100] loss=2.4581 val_loss=2.7079 scale=2.0000 norm=4.3208
== Early stopping achieved.
== Best iteration / VAL143 (val_loss=2.6410)
[iter 0] loss=3.3183 val_loss=3.1696 scale=2.0000 norm=10.7922
== Early stopping achieved.
== Best iteration / VAL4 (val_loss=3.

In [ ]:
y_train

In [70]:
ngb_regressor = ngb.NGBRegressor(Base = b1,
                  Dist = ngb.distns.Normal,
                  Score = ngb.scores.LogScore,
                  n_estimators = best['n_estimators'],
                  learning_rate = best['learning_rate'],
                  minibatch_frac = best['minibatch_frac'],
                  early_stopping_rounds=5).fit(X_train.loc[:, ~X_train.columns.isin(['GAME_DATE', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION'])], np.ravel(y_train))

[iter 0] loss=3.3029 val_loss=3.2884 scale=2.0000 norm=10.6409


In [73]:
Y_preds = ngb_regressor.predict(X_test.loc[:, ~X_test.columns.isin(['GAME_DATE', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION'])])
Y_dists = ngb_regressor.pred_dist(X_test.loc[:, ~X_test.columns.isin(['GAME_DATE', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION'])])

In [75]:
# test Mean Squared Error
test_MSE = mean_squared_error(Y_preds, y_test)
print('Test MSE', test_MSE)

Test MSE 19.662727468819938


In [76]:
np.sqrt(test_MSE)

4.434267410612484

In [ ]:
# test Negative Log Likelihood
test_NLL = -Y_dists.logpdf(Y_test).mean()
print('Test NLL', test_NLL)